In [2]:
from itertools import product
import dolphindb as ddb

In [3]:
%matplotlib inline

In [4]:
ddb_conn = ddb.session()
ddb_conn.connect("localhost", 8848, userid="admin", password="123456")

True

In [5]:
load_table = """
tickQuote = loadTable("dfs://quote_comp", `tickQuote) //委託簿
tickTSE = loadTable("dfs://daily_tse", `tickTSE)      //成交行情
tickTFE = loadTable("dfs://daily_tfe", `tickTFE)      //成交行情
tickIDX = loadTable("dfs://daily_index", `tickIDX)    //指數
"""

In [6]:
ddb_conn.run(load_table)

In [105]:
df_free = ddb_conn.run("select sum(freeSize) as freeSize , sum(blockSize) as allocatedSize from mem();")
print(df_free["freeSize"]/ df_free["allocatedSize"])
df_free

0    0.997941
dtype: float64


,freeSize,allocatedSize
0,104641728,104857600


In [6]:
ddb_conn.run("getSessionMemoryStat()")

,userId,sessionId,memSize
0,admin,2882591513,16
1,admin,2333906441,16


In [12]:
clean_cache = """
undef(all)
clearAllCache()
getSessionMemoryStat()
"""
ddb_conn.run(clean_cache)

,userId,sessionId,memSize
0,admin,2882591513,112


In [7]:
ddb_conn.run("select count(*) from tickTFE group by Date;")

,Date,count
0,2019-03-04,256713
1,2019-03-05,269710
2,2019-03-06,269753
3,2019-03-07,264026
4,2019-03-08,317663
...,...,...
215,2019-11-26,254385
216,2019-11-27,229895
217,2019-11-28,210255
218,2019-11-29,348713


In [8]:
df = ddb_conn.run('select concatDateTime(Date, Time) as Datetime, Close, Volume, TickType, Index '
             'from tickTFE where Code="TXFL9"')
df

,Datetime,Close,Volume,TickType,Index
0,2019-03-04 08:45:00.679,9882.0,1.0,1.0,0
1,2019-03-04 08:45:00.817,9882.0,1.0,1.0,0
2,2019-03-04 08:45:29.059,9871.0,1.0,2.0,0
3,2019-03-04 08:45:45.879,9872.0,1.0,1.0,0
4,2019-03-04 08:45:58.377,9872.0,1.0,2.0,0
...,...,...,...,...,...
399836,2019-11-30 04:59:52.589,11480.0,1.0,2.0,0
399837,2019-11-30 04:59:55.617,11481.0,3.0,1.0,0
399838,2019-11-30 04:59:56.131,11481.0,1.0,1.0,0
399839,2019-11-30 04:59:56.380,11480.0,3.0,2.0,0


In [33]:
ddb_conn.run('select concatDateTime(Date, Time) as Datetime, * '
             'from tickQuote where Code="TXFL9", Date=2019.11.20')

,Datetime,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskVolume1,AskVolume2,AskVolume3,AskVolume4,...,BidPrice4,BidPrice5,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5,Date,Time,Code
0,2019-11-20 00:00:00.125,11624.0,11625.0,11626.0,11627.0,11628.0,10.0,25.0,7.0,5.0,...,11620.0,11619.0,2.0,24.0,14.0,58.0,8.0,2019-11-20,1970-01-01 00:00:00.125,TXFL9
1,2019-11-20 00:00:00.500,11624.0,11625.0,11626.0,11627.0,11628.0,10.0,25.0,7.0,5.0,...,11620.0,11619.0,2.0,24.0,14.0,58.0,8.0,2019-11-20,1970-01-01 00:00:00.500,TXFL9
2,2019-11-20 00:00:00.750,11624.0,11625.0,11626.0,11627.0,11628.0,10.0,25.0,7.0,5.0,...,11620.0,11619.0,2.0,24.0,14.0,58.0,8.0,2019-11-20,1970-01-01 00:00:00.750,TXFL9
3,2019-11-20 00:00:00.875,11624.0,11625.0,11626.0,11627.0,11628.0,10.0,25.0,7.0,5.0,...,11620.0,11619.0,2.0,24.0,14.0,58.0,8.0,2019-11-20,1970-01-01 00:00:00.875,TXFL9
4,2019-11-20 00:00:01.000,11624.0,11625.0,11626.0,11627.0,11628.0,10.0,25.0,7.0,5.0,...,11620.0,11619.0,2.0,24.0,14.0,58.0,8.0,2019-11-20,1970-01-01 00:00:01.000,TXFL9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199131,2019-11-20 23:59:57.744,11598.0,11599.0,11600.0,11601.0,11602.0,12.0,33.0,37.0,31.0,...,11594.0,11593.0,12.0,18.0,37.0,27.0,50.0,2019-11-20,1970-01-01 23:59:57.744,TXFL9
199132,2019-11-20 23:59:58.119,11598.0,11599.0,11600.0,11601.0,11602.0,12.0,33.0,32.0,31.0,...,11594.0,11593.0,11.0,18.0,32.0,27.0,50.0,2019-11-20,1970-01-01 23:59:58.119,TXFL9
199133,2019-11-20 23:59:59.244,11598.0,11599.0,11600.0,11601.0,11602.0,12.0,33.0,32.0,31.0,...,11594.0,11593.0,11.0,18.0,32.0,27.0,50.0,2019-11-20,1970-01-01 23:59:59.244,TXFL9
199134,2019-11-20 23:59:59.369,11598.0,11599.0,11600.0,11601.0,11602.0,12.0,33.0,32.0,31.0,...,11594.0,11593.0,11.0,18.0,32.0,27.0,50.0,2019-11-20,1970-01-01 23:59:59.369,TXFL9


In [57]:
quote_cols = [('').join(c) for c in product(["Ask", "Bid"], 
                                            ["Price", "Volume"], 
                                            map(str, range(1, 6)))]
quote_cols_str = (", ").join(quote_cols)

In [58]:
ddb_conn.run('select concatDateTime(Date,Time) as DateTime, Code, {0} '
             'from tickQuote where Date=2019.11.09 and Code="TXFK9";'.format(quote_cols_str))

,DateTime,Code,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskVolume1,AskVolume2,AskVolume3,...,BidPrice1,BidPrice2,BidPrice3,BidPrice4,BidPrice5,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5
0,2019-11-09 00:00:00.090,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,26.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,24.0,34.0,32.0,46.0,40.0
1,2019-11-09 00:00:00.840,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,24.0,34.0,37.0,46.0,40.0
2,2019-11-09 00:00:00.965,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,34.0,37.0,46.0,40.0
3,2019-11-09 00:00:01.465,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,39.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,34.0,38.0,46.0,40.0
4,2019-11-09 00:00:01.715,TXFK9,11586.0,11587.0,11588.0,11589.0,11590.0,8.0,37.0,31.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,25.0,36.0,38.0,46.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21773,2019-11-09 04:59:55.340,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,9.0,2.0,7.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,10.0,5.0,5.0,7.0,9.0
21774,2019-11-09 04:59:55.965,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11585.0,11584.0,11583.0,11582.0,11581.0,10.0,5.0,5.0,7.0,9.0
21775,2019-11-09 04:59:56.715,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11586.0,11585.0,11584.0,11583.0,11582.0,2.0,10.0,5.0,5.0,7.0
21776,2019-11-09 04:59:57.215,TXFK9,11587.0,11588.0,11589.0,11590.0,11591.0,8.0,2.0,7.0,...,11586.0,11585.0,11584.0,11583.0,11582.0,2.0,10.0,5.0,5.0,7.0


In [63]:
ddb_conn.run('select count(*) from tickQuote where Date=2019.11.08 group by Code;').sort_values(
    'count', ascending=False)

,Code,count
669,MXFK9,227380
31,TXFK9,216979
45,UNFC0,175254
320,UNFL9,175089
567,MX2K9,168135
...,...,...
9745,HCFL9/C0,1
20859,KWFL9/C0,1
17221,FWFL9/C0,1
8614,DIFL9/I0,1


In [27]:
ddb_conn.run('select * from tickTSE where Code="2330"')

,Code,Date,Time,Close,Volume,TickType,Index
0,2330,2019-03-04,1970-01-01 09:00:00.842,239.5,5082.0,0.0,0
1,2330,2019-03-04,1970-01-01 09:00:05.873,239.5,54.0,0.0,0
2,2330,2019-03-04,1970-01-01 09:00:10.901,239.5,11.0,1.0,0
3,2330,2019-03-04,1970-01-01 09:00:15.926,239.5,32.0,1.0,0
4,2330,2019-03-04,1970-01-01 09:00:20.949,239.5,8.0,1.0,0
...,...,...,...,...,...,...,...
452859,2330,2019-11-22,1970-01-01 13:24:49.692,308.0,4.0,2.0,0
452860,2330,2019-11-22,1970-01-01 13:24:54.706,308.5,3.0,1.0,0
452861,2330,2019-11-22,1970-01-01 13:24:59.720,308.5,4.0,1.0,0
452862,2330,2019-11-22,1970-01-01 13:30:00.000,309.0,4348.0,1.0,0


In [6]:
import requests
import pandas as pd

In [7]:
res = requests.get("https://www.taifex.com.tw/cht/9/futuresQADetail")

In [8]:
df_stock_value_rank = pd.read_html(res.text)[0]
df_stock_value_rank.columns = ['rank', 'code', 'name', 'v'] + ['rank1', 'code1', 'name1', 'v1']
df_stock_value_rank['code'][:10].tolist()

[2330, 2317, 6505, 2412, 2454, 1301, 3008, 1303, 2882, 1326]

In [9]:
ddb_conn.run('select concatDateTime(Date, Time) as Datetime, Code, Close, Volume, TickType, Index '
             'from tickTSE where {}'.format((" or ").join([
                f'Code=="{code}"' for code in df_stock_value_rank['code'][:10].tolist()]))
            )

,Datetime,Code,Close,Volume,TickType,Index
0,2019-03-04 09:00:00.842,2330,239.5,5082.0,0.0,0
1,2019-03-04 09:00:05.873,2330,239.5,54.0,0.0,0
2,2019-03-04 09:00:10.901,2330,239.5,11.0,1.0,0
3,2019-03-04 09:00:15.926,2330,239.5,32.0,1.0,0
4,2019-03-04 09:00:20.949,2330,239.5,8.0,1.0,0
...,...,...,...,...,...,...
2641494,2019-11-22 13:24:41.900,6505,96.7,1.0,2.0,0
2641495,2019-11-22 13:24:51.929,6505,96.7,1.0,2.0,0
2641496,2019-11-22 13:24:56.944,6505,96.7,1.0,2.0,0
2641497,2019-11-22 13:30:00.000,6505,97.0,366.0,1.0,0


In [10]:
ddb_conn.run("""
txfk9tick = select * from tickTFE where Code="TXFK9"
txfk9quote = select * from tickQuote where Code="TXFK9"
select concatDateTime(Date, Time) as Datetime, concatDateTime(Date, txfk9quote.Time) as DateTimeQuote, AskPrice1, AskVolume1, BidPrice1, BidVolume1, Close, Volume, Index, TickType  from aj(txfk9tick, txfk9quote, `Date`Time)
""")

,Datetime,DateTimeQuote,AskPrice1,AskVolume1,BidPrice1,BidVolume1,Close,Volume,Index,TickType
0,2019-08-22 09:00:35.832,2019-08-22 09:00:35.797,10498.0,3.0,10494.0,3.0,10496.0,1.0,0,1.0
1,2019-08-22 09:01:09.919,2019-08-22 09:01:09.797,10500.0,2.0,10497.0,3.0,10500.0,1.0,0,1.0
2,2019-08-22 09:01:16.597,2019-08-22 09:01:16.547,10500.0,1.0,10497.0,3.0,10500.0,1.0,0,1.0
3,2019-08-22 09:01:47.446,2019-08-22 09:01:47.421,10499.0,3.0,10496.0,1.0,10496.0,1.0,0,2.0
4,2019-08-22 09:02:55.127,2019-08-22 09:02:55.046,10501.0,3.0,10496.0,3.0,10498.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
844448,2019-11-20 13:29:59.729,2019-11-20 13:29:59.648,11608.0,839.0,11607.0,3508.0,11607.0,2.0,0,2.0
844449,2019-11-20 13:29:59.769,2019-11-20 13:29:59.648,11608.0,839.0,11607.0,3508.0,11608.0,4.0,0,1.0
844450,2019-11-20 13:29:59.928,2019-11-20 13:29:59.898,11608.0,835.0,11607.0,3505.0,11607.0,1.0,0,2.0
844451,2019-11-20 13:29:59.933,2019-11-20 13:29:59.898,11608.0,835.0,11607.0,3505.0,11607.0,3.0,0,2.0


In [3]:
sess = ddb.session()
sess.connect("localhost", 8868, userid="admin", password="123456")

True

In [4]:
LOAD_TABLE = """
tickTFE = loadTable("dfs://daily_tfe", `tickTFE)
"""
sess.run(LOAD_TABLE)

In [6]:
df = sess.run("""
select * from tickTFE where Date=2016.10.18
""")

In [7]:
df

,Code,Date,Time,Close,Volume
0,TX201610,2016-10-18,1970-01-01 08:45:00.000,9183.0,774.0
1,TX201610,2016-10-18,1970-01-01 08:45:00.000,9182.0,2.0
2,TX201610,2016-10-18,1970-01-01 08:45:00.000,9182.0,2.0
3,TX201610,2016-10-18,1970-01-01 08:45:00.000,9182.0,2.0
4,TX201610,2016-10-18,1970-01-01 08:45:00.000,9182.0,2.0
...,...,...,...,...,...
59265,TX201610,2016-10-18,1970-01-01 13:44:05.900,9210.0,2.0
59266,TX201610,2016-10-18,1970-01-01 13:44:05.900,9211.0,2.0
59267,TX201610,2016-10-18,1970-01-01 13:44:05.900,9210.0,2.0
59268,TX201610,2016-10-18,1970-01-01 13:44:05.900,9210.0,4.0


In [12]:
df = sess.run("""
select Code, concatDateTime(Date, Time) as DateTime, Close, Volume from tickTFE where minute(Time)=13:30m
""")

In [13]:
df

,Code,DateTime,Close,Volume
0,TX201101,2011-01-03 13:30:00.000,9020.0,2.0
1,TX201101,2011-01-03 13:30:00.000,9020.0,4.0
2,TX201101,2011-01-03 13:30:00.000,9020.0,4.0
3,TX201101,2011-01-03 13:30:00.000,9020.0,2.0
4,TX201101,2011-01-03 13:30:00.000,9020.0,4.0
...,...,...,...,...
835210,TX201902,2019-02-19 13:30:05.900,10144.0,2.0
835211,TX201902,2019-02-19 13:30:05.900,10144.0,2.0
835212,TX201902,2019-02-19 13:30:05.900,10144.0,2.0
835213,TX201902,2019-02-19 13:30:05.900,10144.0,2.0
